In [1]:
%reload_ext autoreload
%autoreload 2

In [4]:
from depsurf import DepReport, Dep, DepKind
from utils import load_pkl
from typing import Dict

report_data: Dict[str, Dict[Dep, DepReport]] = load_pkl("bcc")

# tool_name = "biotop"
# tool_name = "readahead"

# if tool_name == "biotop":
KEYS = {
    "biotop": [
        DepKind.FUNC("blk_account_io_start"),
        DepKind.FUNC("blk_account_io_done"),
        DepKind.FUNC("__blk_account_io_start"),
        DepKind.FUNC("__blk_account_io_done"),
        DepKind.TRACEPOINT("block_io_start"),
        DepKind.TRACEPOINT("block_io_done"),
        DepKind.FIELD("request::rq_disk"),
        DepKind.FIELD("request_queue::disk"),
    ],
    # ]
    # elif tool_name == "readahead":
    # KEYS = [
    "readahead": [
        DepKind.FUNC("__do_page_cache_readahead"),
        DepKind.FUNC("do_page_cache_ra"),
        DepKind.FUNC("__page_cache_alloc"),
    ],
}

reports = {}

for tool_name, keys in KEYS.items():
    for key in keys:
        reports[key] = report_data[tool_name][key]

[          pkl.py:18 ] INFO: Loding bcc from /Users/szhong/Downloads/bpf-study/output/bcc.pkl


In [3]:
import matplotlib.pyplot as plt
from matplotlib import colors
import numpy as np

from depsurf import VersionGroup, Consequence
from utils import save_fig


fig, ax = plt.subplots(figsize=(9, 9))
ax: plt.Axes

data = np.array(
    [[colors.to_rgb(val.color) for val in report.values] for report in reports.values()]
)

xlabels = (
    VersionGroup.REGULAR.version_labels
    + VersionGroup.ARCH.version_labels
    + VersionGroup.REGULAR.version_labels[:-1]
    + VersionGroup.ARCH.version_labels
)


ax.set_xticks(
    np.arange(data.shape[1]),
    labels=xlabels,
    rotation=90,
    fontsize=9,
)
ax.set_yticks(
    np.arange(data.shape[0]), labels=[d.name for d in reports.keys()], fontsize=9
)
ax.set_xticks(np.arange(data.shape[1] + 1) - 0.5, minor=True)
ax.set_yticks(np.arange(data.shape[0] + 1) - 0.5, minor=True)
ax.imshow(data)

ax.grid(which="minor", color="white", linestyle="-", linewidth=0.1)
ax.xaxis.set_ticks_position("top")
ax.xaxis.set_label_position("top")
ax.spines["top"].set_visible(True)
ax.spines["bottom"].set_visible(False)
ax.tick_params(which="minor", length=0)


def plot_secondary_ticks(
    ax: plt.Axes,
    axis_name,
    lengths,
    labels,
    pad,
    rotation=0,
    linewidth=0.5,
    linestyle="--",
):
    val = np.array(list(lengths))
    cum = np.cumsum(val)
    mid = cum - val / 2 - 0.5
    div = np.array([0, *cum][:-1]) - 0.5

    if axis_name == "x":
        axis = ax.secondary_xaxis("top").xaxis
    else:
        axis = ax.secondary_yaxis("left").yaxis

    # plot labels
    axis.set_ticks(mid, labels, rotation=rotation, verticalalignment="center")
    axis.set_tick_params(length=0, pad=pad)

    # plot ticks
    axis.set_ticks(div, labels=[], minor=True)
    axis.set_tick_params(length=pad + 10, width=linewidth, which="minor")

    axline = ax.axvline if axis_name == "x" else ax.axhline
    for d in div:
        axline(d, color="black", lw=linewidth, linestyle=linestyle)


plot_secondary_ticks(
    ax,
    "x",
    [
        len(VersionGroup.REGULAR),
        len(VersionGroup.ARCH),
        VersionGroup.REGULAR.num_pairs,
        VersionGroup.ARCH.num_pairs,
    ],
    ["Version", "Arch", "Version Diff", "Arch Diff"],
    pad=45,
)

plot_secondary_ticks(
    ax,
    "y",
    [4, 2, 2, 3],
    ["Func", "TP", "Field", "Func"],
    pad=140,
    rotation=90,
)

plot_secondary_ticks(
    ax,
    "y",
    [8, 3],
    ["biptop", "read\nahead"],
    linewidth=1,
    linestyle="-",
    pad=160,
    rotation=90,
)


for i, report in enumerate(reports.values()):
    for j, val in enumerate(report.values):
        ax.text(
            j,
            i,
            val.text,
            ha="center",
            va="center",
            color="white",
            fontsize=8,
        )

consequences = [c for c in Consequence if c != Consequence.UNKNOWN]

CONSEQUENCES = {
    Consequence.MISS: "Incomplete result",
    Consequence.STRAY: "Incorrect result",
    Consequence.ERROR: "Explicit error",
    Consequence.OK: "OK",
}

ax.legend(
    handles=[plt.Rectangle((0, 0), 1, 1, fc=c.color) for c in CONSEQUENCES],
    labels=list(CONSEQUENCES.values()),
    loc="upper left",
    bbox_to_anchor=(-0.275, 1.45),
    ncol=1,
    fontsize=8,
)


save_fig(fig, "report", close=True)

[          mpl.py:84 ] INFO: Saved figure to /Users/szhong/Downloads/bpf-study/paper/figs/report.pdf
